In [ ]:
import avstack
import avapi
from tqdm import tqdm

data_dir_sensor = '/data/spencer/CARLA/multi-sensor-v1/'
# data_dir_agent  = '/data/spencer/CARLA/multi-agent-v1/'

CSM_sensor = avapi.carla.CarlaScenesManager(data_dir=data_dir_sensor)
# CSM_agent = avapi.carla.CarlaScenesManager(data_dir=data_dir_agent)

CSM = CSM_sensor
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes['val'][2])

In [ ]:
def show_image_boxes_frame(frame, sensor):
    if 'depth' in sensor.lower():
        func = CDM.get_depth_image
    elif 'semseg' in sensor.lower():
        func = CDM.get_semseg_image
    else:
        func = CDM.get_image
    avapi.visualize.snapshot.show_image_with_boxes(
        img=func(frame=frame, sensor=sensor),
        boxes=CDM.get_objects(frame=frame, sensor=sensor, max_occ=avstack.environment.objects.Occlusion.MOST),
        inline=True
    )

def show_lidar_boxes(frame, lidar_sensor, camera_sensor):
    img = CDM.get_image(frame=frame, sensor=camera_sensor)
    pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor)
    objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, max_occ=avstack.environment.objects.Occlusion.MOST)
    avapi.visualize.snapshot.show_lidar_on_image(pc, img, boxes=objs)
    avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objs)

In [ ]:
frame = CDM.frames[10]

ctype = "BACK"
cam1 = "CAM_{}".format(ctype)
cam2 = "CAM_{}_SEMSEG".format(ctype)
cam3 = "CAM_{}_DEPTH".format(ctype)
pc1  = "LIDAR_TOP"

pc = CDM.get_lidar(frame=frame, sensor=pc1)

MOST = avstack.environment.objects.Occlusion.MOST

show_image_boxes_frame(frame, cam1)
show_image_boxes_frame(frame, cam2)
show_image_boxes_frame(frame, cam3)
show_lidar_boxes(frame, pc1, cam1)

## Run Perception

#### 2D

In [ ]:
M2D_vehicle = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='fasterrcnn', dataset='carla', epoch='9', threshold=0.7,
)

In [ ]:
frame = CDM.frames[10]

cam1 = "CAM_FRONT"
img = CDM.get_image(frame=frame, sensor=cam1)
objs = CDM.get_objects(frame=frame, sensor=cam1, max_occ=avstack.environment.objects.Occlusion.PARTIAL)
dets = M2D_vehicle(data=img)

avapi.visualize.snapshot.show_image_with_boxes(img, boxes=dets, inline=True)

#### 3D

In [ ]:
M3D_vehicle = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='pointpillars', dataset='carla', epoch='latest', threshold=0.5,
)

In [ ]:
data_dir_lidar = '/data/spencer/CARLA/ego-lidar/'
CSM_lidar = avapi.carla.CarlaScenesManager(data_dir=data_dir_lidar)
CDM = CSM_lidar.get_scene_dataset_by_name(CSM_lidar.splits_scenes['val'][3])
frame = CDM.frames[10]

cam1 = "CAM_FRONT"
img = CDM.get_image(frame=frame, sensor=cam1)
pc = CDM.get_lidar(frame=frame, sensor="LIDAR_TOP")
objs = CDM.get_objects(frame=frame, sensor="LIDAR_TOP", max_occ=avstack.environment.objects.Occlusion.MOST)
dets = M3D_vehicle(data=pc)
avapi.visualize.snapshot.show_image_with_boxes(img, boxes=dets, inline=True)

In [ ]:
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objs)
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=dets)

## Compute Model AP

In [ ]:
M3D_vehicle = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='pointpillars', dataset='carla', epoch='latest', threshold=0.1,
)
data_dir_lidar = '/data/spencer/CARLA/ego-lidar/'
CSM_lidar = avapi.carla.CarlaScenesManager(data_dir=data_dir_lidar)
CDM = CSM_lidar.get_scene_dataset_by_name(CSM_lidar.splits_scenes['val'][3])

In [ ]:
results = []
i_start = 5
n_frames = 20
for frame in tqdm( CDM.get_frames(sensor="main_lidar")[i_start:n_frames+i_start]):
    try:
        pc = CDM.get_lidar(frame=frame, sensor="main_lidar")
        objs = CDM.get_objects(frame=frame, sensor="main_lidar")
    except FileNotFoundError:
        continue
    dets = M3D_vehicle(data=pc)
    res = avapi.evaluation.ResultManager(idx=frame, detections=dets, truths=objs, threshold=0.3)
    results.append(res)

In [ ]:
ap, mprec, mrec = avapi.evaluation.metrics.get_ap_from_results(results, by_class=True)
lamr, mr, fppi = avapi.evaluation.metrics.get_lamr_from_results(results, by_class=True)

In [ ]:
avapi.evaluation.metrics.plot_all_results_statistics(results, by_class=True)

In [ ]:
avapi.evaluation.metrics.barplot_ap(ap)
avapi.evaluation.metrics.barplot_lamr(lamr)

In [ ]:
import pickle
import numpy as np
with open ('../submodules/lib-avstack-core/third_party/mmdetection3d/data/carla/ego-lidar/val_annotation_carla_in_nuscenes.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
for dl in [data['data_list'][20]]:
    for instance in dl['instances']:
        if instance['distance'] < 10:
            print(instance)

In [ ]:
CSM = CSM_sensor
for name in CSM.splits_scenes['train']:
    CDM = CSM.get_scene_dataset_by_name(name)
    frame = CDM.frames[10]
    img = CDM.get_image(frame=frame, sensor="main_camera")
    img.view()